In [1]:
import pandas as pd

import tensorflow as tf
import json
import numpy as np
import pandas as pd
import cv2

import matplotlib.pyplot as plt
import seaborn as sns

from cell_division.nets.transfer_learning import CNN
from auxiliary.data.dataset_cell import CellDataset
from auxiliary.data.dataset_unlabeled import UnlabeledDataset
from auxiliary import values as v
from auxiliary.utils.colors import bcolors as c
from auxiliary.utils import visualizer as vis

from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.layers import GlobalAveragePooling2D
from cell_division.nets.custom_layers import (
    w_cel_loss, 
    focal_loss,
    ExtendedLSEPooling,
    extended_w_cel_loss,
    LSEPooling
)

from cell_division.nets.cam import GradCAM, overlay_heatmap, CAM, GradCAMpp
from cell_division.semi_supervised import semi_supervised_learning as SSL

# GPU config
from auxiliary.utils.timer import LoadingBar
from auxiliary.gpu.gpu_tf import (
    increase_gpu_memory, 
    set_gpu_allocator, 
    clear_session
)

increase_gpu_memory()
# set_gpu_allocator()

ValueError: Cannot set memory growth on device when virtual devices configured

In [2]:
img_dir = v.data_path + 'CellDivision/images_nuclei/'
img_dir_unlabeled = v.data_path + 'CellDivision/images_unlabeled/'

label_train_dir = v.data_path + 'CellDivision/undersampled/train.csv'
label_test_dir = v.data_path + 'CellDivision/undersampled/test.csv'
label_val_dir = v.data_path + 'CellDivision/undersampled/val.csv'

INPUT_SHAPE = (100, 100, 3)
BATCH_SIZE = 128

In [3]:
train_generator = CellDataset(
    img_dir, 
    label_train_dir,
    batch_size=BATCH_SIZE,
    resize=INPUT_SHAPE[:2]
)

val_generator = CellDataset(
    img_dir, 
    label_val_dir,
    batch_size=BATCH_SIZE,
    resize=INPUT_SHAPE[:2]
)

test_generator = CellDataset(
    img_dir, 
    label_test_dir,
    batch_size=BATCH_SIZE,
    resize=INPUT_SHAPE[:2]
)

unlabeled_generator = UnlabeledDataset(
    img_dir_unlabeled,
    batch_size=1,
    resize=INPUT_SHAPE[:2]
)

In [4]:
model = CNN(
    base=tf.keras.applications.VGG16,
    input_shape=INPUT_SHAPE,
    n_classes=3
)
model.build_top(activation='softmax', b_type='CAM', pooling=ExtendedLSEPooling)
model.compile(
    lr=.001,
    loss=extended_w_cel_loss()
)

In [ ]:
model = SSL(
    model, 
    train_generator, val_generator, test_generator,
    unlabeled_generator,
    max_iter=10,
    verbose=1
)
model.model.save('../models/cellular_division_models/vgg16_semi.h5')

Iteration: 1
	Pre-training...
Epoch 1/100
4/4 [==============================] - 11s 3s/step - loss: 1.2200 - auc: 0.9249 - val_loss: 1.1342 - val_auc: 0.8701 - lr: 1.0000e-07
Epoch 2/100
4/4 [==============================] - 11s 3s/step - loss: 1.2191 - auc: 0.9213 - val_loss: 1.1211 - val_auc: 0.8782 - lr: 1.0000e-07
Epoch 3/100
4/4 [==============================] - 11s 3s/step - loss: 1.3280 - auc: 0.9216 - val_loss: 1.1080 - val_auc: 0.8853 - lr: 1.0000e-07
Epoch 4/100
4/4 [==============================] - ETA: 0s - loss: 1.2867 - auc: 0.9239
Epoch 4: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-08.
4/4 [==============================] - 11s 3s/step - loss: 1.2867 - auc: 0.9239 - val_loss: 1.0947 - val_auc: 0.8922 - lr: 1.0000e-07
Epoch 5/100
4/4 [==============================] - 12s 3s/step - loss: 1.1158 - auc: 0.9315 - val_loss: 1.0813 - val_auc: 0.8983 - lr: 1.0000e-08
Epoch 6/100
4/4 [==============================] - 12s 4s/step - loss: 1.2693 - auc: 0.9

In [ ]:
model.model.evaluate(test_generator)

In [ ]:
pred = model.model.predict(test_generator).round().astype(int)

print(
    classification_report(
        test_generator.img_labels, [test_generator.oh2class(p) for p in pred], 
        target_names=test_generator.CLASS_NAMES,
        zero_division=0
    )
)

In [ ]:
cf_matrix = confusion_matrix(
   test_generator.img_labels, [test_generator.oh2class(p) for p in pred]
)

plt.figure(figsize=(12, 12))
vis.plot_confusion_matrix(cf_matrix)